In [1]:
import pandas as pd
from datetime import datetime
import fsspec
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [14]:
import pandas as pd

sparkDF = spark.sql("SELECT * FROM bronze_lakehouse.green_tripdata_2017")
green_taxi_data = sparkDF.toPandas()

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 16, Finished, Available, Finished)

In [10]:
# Convert date-time columns to datetime objects
green_taxi_data['lpep_pickup_datetime'] = pd.to_datetime(green_taxi_data['lpep_pickup_datetime'])
green_taxi_data['lpep_dropoff_datetime'] = pd.to_datetime(green_taxi_data['lpep_dropoff_datetime'])

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 12, Finished, Available, Finished)

In [15]:
display(green_taxi_data)

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 05b4050e-dfdc-40cf-8477-84f2140d04e9)

In [18]:
# Filter out trips with zero fare amount
green_taxi_data = green_taxi_data[green_taxi_data['fare_amount'] > 0]
display(green_taxi_data)

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 00ef63db-b22e-4c6c-92d4-49a4a2b1e28b)

In [19]:

# Remove rows with zero passenger counts
green_taxi_data = green_taxi_data[green_taxi_data['passenger_count'] > 0]

# Filter out trips with zero trip distance
green_taxi_data = green_taxi_data[green_taxi_data['trip_distance'] > 0]

# Filter out trips with zero fare amount
green_taxi_data = green_taxi_data[green_taxi_data['fare_amount'] > 0]

# Convert categorical variables to appropriate data types
green_taxi_data['VendorID'] = green_taxi_data['VendorID'].astype('category')
green_taxi_data['store_and_fwd_flag'] = green_taxi_data['store_and_fwd_flag'].astype('category')
green_taxi_data['RatecodeID'] = green_taxi_data['RatecodeID'].astype('category')
green_taxi_data['payment_type'] = green_taxi_data['payment_type'].astype('category')

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 21, Finished, Available, Finished)

In [20]:
cleaned_green_taxi_data=green_taxi_data
print(cleaned_green_taxi_data)

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 22, Finished, Available, Finished)

         VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0               2  2017-05-18 16:55:43   2017-05-18 18:04:11   
2               1  2017-12-01 18:55:10   2017-12-01 19:39:05   
3               1  2017-06-23 12:34:57   2017-06-23 13:42:08   
4               2  2017-03-03 17:26:03   2017-03-03 18:47:43   
5               2  2017-05-15 16:15:15   2017-05-15 18:14:12   
...           ...                  ...                   ...   
11736901        2  2017-02-24 14:53:53   2017-02-24 14:59:12   
11736902        2  2017-02-24 15:25:00   2017-02-24 15:30:24   
11736903        2  2017-02-25 15:27:34   2017-02-25 15:33:30   
11736904        2  2017-02-27 14:35:13   2017-02-27 14:40:24   
11736905        2  2017-02-28 08:24:34   2017-02-28 08:29:35   

         store_and_fwd_flag RatecodeID  PULocationID  DOLocationID  \
0                         N          2           130            48   
2                         N          2            41           132   
3                    

In [21]:
spark_clean_data = spark.createDataFrame(cleaned_green_taxi_data)

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 23, Finished, Available, Finished)

In [22]:
# write to the silver layer
spark_clean_data.write.mode("overwrite").format('delta').saveAsTable("silver_lakehouse.green_tripdata_2017")

StatementMeta(, eca6cd4f-cba5-4cdd-9a5a-db553dd7dd23, 24, Finished, Available, Finished)

In [5]:
import pandas as pd
def bronze_to_silver(table_name):
    sparkDF = spark.sql("SELECT * FROM bronze_lakehouse." + table_name)
    green_taxi_data = sparkDF.toPandas()
    # Convert date-time columns to datetime objects
    green_taxi_data['lpep_pickup_datetime'] = pd.to_datetime(green_taxi_data['lpep_pickup_datetime'])
    green_taxi_data['lpep_dropoff_datetime'] = pd.to_datetime(green_taxi_data['lpep_dropoff_datetime'])
    # Filter out trips with zero fare amount
    green_taxi_data = green_taxi_data[green_taxi_data['fare_amount'] > 0]
    
    # Remove rows with zero passenger counts
    green_taxi_data = green_taxi_data[green_taxi_data['passenger_count'] > 0]

    # Filter out trips with zero trip distance
    green_taxi_data = green_taxi_data[green_taxi_data['trip_distance'] > 0]

    # Filter out trips with zero fare amount
    green_taxi_data = green_taxi_data[green_taxi_data['fare_amount'] > 0]

    # Convert categorical variables to appropriate data types
    green_taxi_data['VendorID'] = green_taxi_data['VendorID'].astype('category')
    green_taxi_data['store_and_fwd_flag'] = green_taxi_data['store_and_fwd_flag'].astype('category')
    green_taxi_data['RatecodeID'] = green_taxi_data['RatecodeID'].astype('category')
    green_taxi_data['payment_type'] = green_taxi_data['payment_type'].astype('category')
    cleaned_green_taxi_data=green_taxi_data
    spark_clean_data = spark.createDataFrame(cleaned_green_taxi_data)
    # write to the silver layer
    spark_clean_data.write.mode("overwrite").format('delta').saveAsTable("silver_lakehouse."+table_name)

StatementMeta(, 74bfabb4-8c69-4b95-aad3-411bf60e3bc9, 7, Finished, Available, Finished)

In [6]:
# now let's process all other tables
for year in range(2018, 2023):
    bronze_to_silver("green_tripdata_"+str(year))
    display("green_tripdata_"+str(year))


StatementMeta(, 74bfabb4-8c69-4b95-aad3-411bf60e3bc9, 8, Finished, Available, Finished)

'green_tripdata_2018'

'green_tripdata_2019'

'green_tripdata_2020'

'green_tripdata_2021'

'green_tripdata_2022'